# Pricing a Call Option with the Binomial Model

In [93]:
using CSV
using DelimitedFiles, DataFrames
using Statistics
import PyPlot as plt
using Plots
using GLM
using ShiftedArrays
using PyCall
using PlotlyJS

In [94]:
GOOG         = DataFrame(CSV.File("DATA/GOOG.csv"))
BNTX         = DataFrame(CSV.File("DATA/BNTX.csv"))
BNTX_6MONTHS = DataFrame(CSV.File("DATA/BNTX_6MONTHS.csv"))

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2021-09-22,342.803,344.57,336.69,339.26,339.26,1480000
2,2021-09-23,343.0,360.0,341.44,352.99,352.99,2824400
3,2021-09-24,350.0,350.92,332.4,333.48,333.48,2603100
4,2021-09-27,335.361,338.5,305.51,307.09,307.09,4089300
5,2021-09-28,293.7,296.9,275.0,276.52,276.52,5854400
6,2021-09-29,287.0,292.81,275.58,277.44,277.44,2776300
7,2021-09-30,278.14,282.5,268.83,272.99,272.99,2632600
8,2021-10-01,251.0,259.24,230.03,254.79,254.79,6885400
9,2021-10-04,234.93,252.37,233.5,248.9,248.9,3719000


In [95]:
p = PlotlyJS.plot(candlestick(x    =GOOG[!, "Date" ],
                 open =GOOG[!, "Open" ],
                 high =GOOG[!, "High" ],
                 low  =GOOG[!, "Low"  ],
                 close=GOOG[!, "Close"] ),
    Layout(title="GOOG Prices",
           yaxis_title="GOOG Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [96]:
PlotlyJS.savefig(p, "figures/GOOG.html")

"figures/GOOG.html"

In [97]:
p = PlotlyJS.plot(candlestick(
        x    =BNTX[!, "Date" ],
        open =BNTX[!, "Open" ],
        high =BNTX[!, "High" ],
        low  =BNTX[!, "Low"  ],
        close=BNTX[!, "Close"] ),
    Layout(title="BioNTech Prices",
           yaxis_title="BNTX Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [98]:
PlotlyJS.savefig(p, "figures/BNTX.html")

"figures/BNTX.html"

In [99]:
p = PlotlyJS.plot(candlestick(
        x    =BNTX_6MONTHS[!, "Date" ],
        open =BNTX_6MONTHS[!, "Open" ],
        high =BNTX_6MONTHS[!, "High" ],
        low  =BNTX_6MONTHS[!, "Low"  ],
        close=BNTX_6MONTHS[!, "Close"] ),
    Layout(title="BioNTech Prices",
           yaxis_title="BNTX Stock [\$]")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [100]:
PlotlyJS.savefig(p, "figures/BNTX_6MONTHS.html")

"figures/BNTX_6MONTHS.html"

In [101]:
function Return( df )
    df[!,:Return    ] =     ( df.Close - lag(df.Close, 1) ) ./ lag(df.Close, 1)
    df[!,:Return_LOG] = log.( df.Close                      ./ lag(df.Close, 1) )
end

Return( GOOG         )
Return( BNTX         )
Return( BNTX_6MONTHS )
GOOG

,Date,Open,High,Low,Close,Adj Close,Volume,Return
,Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64?
1,2021-12-13,2968.88,2971.25,2927.2,2934.09,2934.09,1205200,missing
2,2021-12-14,2895.4,2908.84,2844.85,2899.41,2899.41,1238900,-0.0118197
3,2021-12-15,2887.32,2950.34,2854.11,2947.37,2947.37,1364000,0.0165414
4,2021-12-16,2961.54,2971.03,2881.85,2896.77,2896.77,1370000,-0.0171679
5,2021-12-17,2854.29,2889.2,2835.76,2856.06,2856.06,2162800,-0.0140536
6,2021-12-20,2813.59,2852.21,2805.0,2848.03,2848.03,1013200,-0.00281158
7,2021-12-21,2863.0,2893.84,2834.7,2884.41,2884.41,977400,0.0127737
8,2021-12-22,2882.0,2946.06,2879.26,2938.98,2938.98,921900,0.018919
9,2021-12-23,2941.79,2971.45,2939.02,2942.85,2942.85,690900,0.00131682


In [102]:
PlotlyJS.plot( GOOG.Date, GOOG.Return,mode="markers+lines",
               Layout(title=      "GOOG Returns",
                      yaxis_title="GOOG Returns") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

In [103]:
PlotlyJS.plot( BNTX.Date, BNTX.Return,mode="markers+lines",
               Layout(title="BioNTech Returns",
                      yaxis_title="BNTX Returns") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

In [104]:
PlotlyJS.plot( BNTX_6MONTHS.Date, BNTX_6MONTHS.Return,mode="markers+lines", name="Returns",
               Layout(title="BioNTech Returns",
                      yaxis_title="BNTX Returns") )

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

## 1-Step Binomial Tree

In [105]:
Year_Days               = 252
Interest_rate3          = 0.0095757 # 22/03/22
Interest_rate6          = 0.0133614 # 22/03/22
Maturity3               = 3/12
Maturity6               = 6/12
S0_GOOG                 = GOOG.Close[length(GOOG.Close)]
S0_BNTX                 = BNTX.Close[length(BNTX.Close)]
S0_BNTX_6MONTHS         = BNTX.Close[length(BNTX.Close)]
K_GOOG                  = 2600
K_BNTX                  = 175
K_BNTX_6MONTHS          = 175

VOL_DAILY_GOOG          = std(skipmissing(GOOG.Return))
VOL_ANNUAL_GOOG         = VOL_DAILY_GOOG * sqrt(Year_Days)

VOL_DAILY_BNTX          = std(skipmissing(BNTX.Return))
VOL_ANNUAL_BNTX         = VOL_DAILY_BNTX * sqrt(Year_Days)

VOL_DAILY_BNTX_6MONTHS  = std(skipmissing(BNTX_6MONTHS.Return))
VOL_ANNUAL_BNTX_6MONTHS = VOL_DAILY_BNTX_6MONTHS * sqrt(Year_Days)

println("Daily volatility of GOOG in the last 3 month:\t", VOL_DAILY_GOOG         )
println("Daily volatility of BNTX in the last 3 month:\t", VOL_DAILY_BNTX         )
println("Daily volatility of BNTX in the last 6 month:\t", VOL_DAILY_BNTX_6MONTHS )
println("\n")

println("Annual volatility of GOOG in the last 3 month:\t", VOL_ANNUAL_GOOG         )
println("Annual volatility of BNTX in the last 3 month:\t", VOL_ANNUAL_BNTX         )
println("Annual volatility of BNTX in the last 6 month:\t", VOL_ANNUAL_BNTX_6MONTHS )
println("\n")

UpDown_GOOG             = ( exp( + VOL_ANNUAL_GOOG         * sqrt( Maturity3 ) ), exp( - VOL_ANNUAL_GOOG         * sqrt( Maturity3 ) ) )
UpDown_BNTX             = ( exp( + VOL_ANNUAL_BNTX         * sqrt( Maturity3 ) ), exp( - VOL_ANNUAL_BNTX         * sqrt( Maturity3 ) ) )
UpDown_BNTX_6MONTHS     = ( exp( + VOL_ANNUAL_BNTX_6MONTHS * sqrt( Maturity6 ) ), exp( - VOL_ANNUAL_BNTX_6MONTHS * sqrt( Maturity6 ) ) )

q_GOOG                  = ( ( 1 + Maturity3 * Interest_rate3 ) - UpDown_GOOG[2] )         / ( UpDown_GOOG[1]         - UpDown_GOOG[2]         )
q_BNTX                  = ( ( 1 + Maturity3 * Interest_rate3 ) - UpDown_BNTX[2] )         / ( UpDown_BNTX[1]         - UpDown_BNTX[2]         )
q_BNTX_6MONTHS          = ( ( 1 + Maturity6 * Interest_rate6 ) - UpDown_BNTX_6MONTHS[2] ) / ( UpDown_BNTX_6MONTHS[1] - UpDown_BNTX_6MONTHS[2] )

PRICE_CALL_GOOG         = (     q_GOOG           * max( 0, S0_GOOG         * UpDown_GOOG[1]         - K_GOOG )           + 
                          ( 1 - q_GOOG         ) * max( 0, S0_GOOG         * UpDown_GOOG[2]         - K_GOOG ) )         / ( 1 + Maturity3 * Interest_rate6 )
PRICE_CALL_BNTX         = (     q_BNTX           * max( 0, S0_BNTX         * UpDown_BNTX[1]         - K_BNTX )           + 
                          ( 1 - q_BNTX         ) * max( 0, S0_BNTX         * UpDown_BNTX[2]         - K_BNTX ) )         / ( 1 + Maturity3 * Interest_rate6 )
PRICE_CALL_BNTX_6MONTHS = (     q_BNTX_6MONTHS   * max( 0, S0_BNTX_6MONTHS * UpDown_BNTX_6MONTHS[1] - K_BNTX_6MONTHS )   + 
                          ( 1 - q_BNTX_6MONTHS ) * max( 0, S0_BNTX_6MONTHS * UpDown_BNTX_6MONTHS[2] - K_BNTX_6MONTHS ) ) / ( 1 + Maturity6 * Interest_rate6 )

println("Price of a Call Option of GOOGLE   with 3 month Maturity:\t", PRICE_CALL_GOOG         )
println("Price of a Call Option of BioNTech with 3 month Maturity:\t", PRICE_CALL_BNTX         )
println("Price of a Call Option of BioNTech with 6 month Maturity:\t", PRICE_CALL_BNTX_6MONTHS )

Daily volatility of GOOG in the last 3 month:	0.02138303844467155
Daily volatility of BNTX in the last 3 month:	0.055035163869881706
Daily volatility of BNTX in the last 6 month:	0.05793839389241427


Annual volatility of GOOG in the last 3 month:	0.3394452119972058
Annual volatility of BNTX in the last 3 month:	0.8736561417803647
Annual volatility of BNTX in the last 6 month:	0.9197434896109904


Price of a Call Option of GOOGLE   with 3 month Maturity:	252.1810610521472
Price of a Call Option of BioNTech with 3 month Maturity:	34.779696221411115
Price of a Call Option of BioNTech with 6 month Maturity:	52.15021589193578


### Multisteps-Binomial Tree

In [106]:
function CALL_PRICE(S, K, r, σ, T, N)
    Δt     = T / N
    U      = exp(σ * √Δt)
    D      = 1 / U
    R      = ( 1 + r * Δt )
    q      = (R - D) / (U - D)

    Payoff = [max(0, S * U^(N-i) * D^i - K ) for i = 0:N]
    
    return sum( Payoff[j+1] * binomial(N,j) * (1-q)^j * (q)^(N-j) for j in 0:N ) / R
end

CALL_PRICE (generic function with 1 method)

In [107]:
print( "\n", CALL_PRICE(S0_GOOG,         K_GOOG,         Interest_rate3, VOL_ANNUAL_GOOG,         Maturity3, 30) )
print( "\n", CALL_PRICE(S0_BNTX,         K_BNTX,         Interest_rate3, VOL_ANNUAL_BNTX,         Maturity3, 30) )
print( "\n", CALL_PRICE(S0_BNTX_6MONTHS, K_BNTX_6MONTHS, Interest_rate6, VOL_ANNUAL_BNTX_6MONTHS, Maturity6, 60) )


210.55087715144228
27.718278175968134
42.30370211207504

In [108]:
GOOG_CALL         = [CALL_PRICE(S0_GOOG,         K_GOOG,         Interest_rate3, VOL_ANNUAL_GOOG,         Maturity3, i) for i in 1:60]
BNTX_CALL         = [CALL_PRICE(S0_BNTX,         K_BNTX,         Interest_rate3, VOL_ANNUAL_BNTX,         Maturity3, i) for i in 1:60]
BNTX_CALL_6MONTHS = [CALL_PRICE(S0_BNTX_6MONTHS, K_BNTX_6MONTHS, Interest_rate6, VOL_ANNUAL_BNTX_6MONTHS, Maturity6, i) for i in 1:60]

BNTX_CALL[60]

27.743059427275632

In [109]:
REAL_CALL_GOOG         = 209.6
#REAL_CALL_BNTX         =  21.7
#REAL_CALL_BNTX_6MONTHS =  14.0 #30.65
REAL_CALL_BNTX         = mean([21.40, 26.40])
REAL_CALL_BNTX_6MONTHS = mean([27.00, 35.60])

31.3

In [110]:
function PlotGOOG_3MONTHS()
    GOOG_PLOT = PlotlyJS.scatter(;x=1:60, y=GOOG_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_GOOG], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Google Call Price, Maturity 3 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([GOOG_PLOT,trace], layout)
end

p = PlotGOOG_3MONTHS()
    

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [111]:
PlotlyJS.savefig(p, "figures/CALL_GOOG.pdf")

"figures/CALL_GOOG.pdf"

In [112]:
function PlotBNTX_3MONTHS()
    BNTX_PLOT = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="BioNTech Call Price, Maturity 3 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BNTX_PLOT,trace], layout)
end

p3 = PlotBNTX_3MONTHS()

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [113]:
PlotlyJS.savefig(p3, "figures/CALL_BNTX.png")

"figures/CALL_BNTX.png"

In [114]:
function PlotBNTX_6MONTHS()
    BNTX_PLOT_6MONTHS = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL_6MONTHS,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX_6MONTHS], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="BioNTech Call Price, Maturity 6 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BNTX_PLOT_6MONTHS,trace], layout)
end

p6 = PlotBNTX_6MONTHS()

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [115]:
PlotlyJS.savefig(p6, "figures/CALL_BNTX_6MONTHS.pdf")

"figures/CALL_BNTX_6MONTHS.pdf"

In [116]:
function PlotBNTX_6MONTHS()
    BNTX_PLOT_6MONTHS = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL_6MONTHS,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX_6MONTHS], 60), mode="lines", name="Mid Price", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Maturity 6 months",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BNTX_PLOT_6MONTHS,trace], layout)
end
function PlotBNTX_3MONTHS()
    BNTX_PLOT = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX], 60), mode="lines", name="Mid Price", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Maturity 3 months",
                       yaxis_title="CALL price [\$]")
    return PlotlyJS.plot([BNTX_PLOT,trace], layout)
end

p = [PlotBNTX_3MONTHS(); PlotBNTX_6MONTHS()]

relayout!(p,height=600,title_text="BioNTech Call Price, Strike ~ S0, as function of number of steps", )

p

data: [
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [117]:
CALL_PRICES_BNTX   = DataFrame(CSV.File("DATA/CALL_BNTX_22032022.csv"))
CALL_PRICES_BNTX_3 = DataFrame(CSV.File("DATA/CALL_BNTX_22032022_3MONTHS.csv"))

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask
,String,String,Float64,Float64,Float64,Float64
1,BNTX220617C00085000,2022-01-21 3:26PM EDT,85.0,68.24,70.7,77.6
2,BNTX220617C00090000,2022-02-22 10:31AM EDT,90.0,65.1,82.0,90.5
3,BNTX220617C00100000,2022-03-17 1:55PM EDT,100.0,68.9,72.8,80.2
4,BNTX220617C00105000,2022-01-24 12:50PM EDT,105.0,46.0,43.8,47.9
5,BNTX220617C00110000,2022-03-15 9:38AM EDT,110.0,49.0,63.5,71.9
6,BNTX220617C00115000,2022-02-23 12:16PM EDT,115.0,38.5,59.1,68.5
7,BNTX220617C00120000,2022-03-21 12:43PM EDT,120.0,58.5,56.0,62.6
8,BNTX220617C00125000,2022-01-03 10:55AM EDT,125.0,119.0,60.6,63.6
9,BNTX220617C00130000,2022-03-17 9:47AM EDT,130.0,44.98,47.0,55.9


In [118]:
PlotlyJS.plot( CALL_PRICES_BNTX.Strike, CALL_PRICES_BNTX[!, Symbol("Volume")] ,mode="markers+lines",
               Layout(title="BioNTech Call option volume as function of the strike",
                      yaxis_title="Volume", xaxis_title="Strike [\$]") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [119]:
BNTX_CALL_6MONTHS_STRIKES = [CALL_PRICE(S0_BNTX_6MONTHS, CALL_PRICES_BNTX.Strike[i],   Interest_rate6, VOL_ANNUAL_BNTX_6MONTHS, Maturity6, 60) for i in 1:length(CALL_PRICES_BNTX.Strike)]
BNTX_CALL_3MONTHS_STRIKES = [CALL_PRICE(S0_BNTX,         CALL_PRICES_BNTX_3.Strike[i], Interest_rate3, VOL_ANNUAL_BNTX        , Maturity3, 60) for i in 1:length(CALL_PRICES_BNTX_3.Strike)]

print("OK")

OK

In [120]:
a = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX.Strike, y=BNTX_CALL_6MONTHS_STRIKES,                 mode="markers+lines", name="60-steps binomial tree prediction")
b = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX.Strike, y=mean([CALL_PRICES_BNTX[!, Symbol("Ask")], CALL_PRICES_BNTX[!, Symbol("Bid")]]), mode="markers+lines", name="Mid price of the option")
layout    = Layout(title="BioNTech Call option prices", yaxis_title="Call prices [\$]", xaxis_title="Strike prices [\$]")
PlotlyJS.plot([a,b], layout)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [121]:
a = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX_3.Strike, y=BNTX_CALL_3MONTHS_STRIKES,                 mode="markers+lines", name="60-steps binomial tree prediction")
b = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX_3.Strike, y=mean([CALL_PRICES_BNTX_3[!, Symbol("Ask")], CALL_PRICES_BNTX_3[!, Symbol("Bid")]]), mode="markers+lines", name="Mid price of the option")
layout    = Layout(title="BioNTech Call option prices", yaxis_title="Call prices [\$]", xaxis_title="Strike prices [\$]")
PlotlyJS.plot([a,b], layout)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [122]:
function PlotBNTX_6MONTHS()
    BNTX_PLOT_6MONTHS = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX.Strike, y=BNTX_CALL_6MONTHS_STRIKES,                 mode="markers+lines", name="Binomial tree prediction 6M")
    trace     = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX.Strike, y=mean([CALL_PRICES_BNTX[!, Symbol("Ask")], CALL_PRICES_BNTX[!, Symbol("Bid")]]), mode="markers+lines", name="Mid price 6M")
    layout    = Layout(title="Maturity 6 months",
                       yaxis_title="CALL price [\$]", xaxis_title="Srike [\$]")
    return PlotlyJS.plot([BNTX_PLOT_6MONTHS,trace], layout)
end
function PlotBNTX_3MONTHS()
    BNTX_PLOT_3MONTHS = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX_3.Strike, y=BNTX_CALL_3MONTHS_STRIKES, mode="markers+lines", name="Binomial tree prediction 3M")
    trace     = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX_3.Strike, y=mean([CALL_PRICES_BNTX_3[!, Symbol("Ask")], CALL_PRICES_BNTX_3[!, Symbol("Bid")]]), mode="markers+lines", name="Mid price 3M")
    layout    = Layout(title="Maturity 3 months",
                       yaxis_title="CALL price [\$]", xaxis_title="Srike [\$]")
    return PlotlyJS.plot([BNTX_PLOT_3MONTHS,trace], layout)
end

p = [PlotBNTX_3MONTHS(); PlotBNTX_6MONTHS()]

data: [
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"